## RNN

Problem: To memorize the (N-2)th digit for a given sequence of binary numbers.

Example: 1 5 2 6 9 2 4 5 6 7 2 

Output: 0 0 1 5 2 6 9 2 4 5 6 


[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]


IndexError: list assignment index out of range

In [174]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize  # For check_grad, approx_fprime

class RNN:
    def __init__ (self, numHidden, numInput, numOutput):
        self.numHidden = numHidden
        self.numInput = numInput
        
        self.U = np.random.randn(numHidden, numHidden) * 1e-1 # 6x6        
        self.V = np.random.randn(numHidden, numInput ) * 1e-1 # 6x1
        self.w = np.random.randn(numHidden, numOutput) * 1e-1 # 6x1
       
        # TODO: IMPLEMENT ME
        # forward prop  
        self.hidden = np.zeros((numHidden, numInput)) # 6x1
        
        
        # backward prop
        self.dU = np.zeros_like(self.U) # 6x6
        self.dw = np.zeros_like(self.w) # 6x1
        self.dV = np.zeros_like(self.V) # 6x1
        self.dhnext = np.zeros_like(self.hidden)
        
        # hyperparameters
        self.learning_rate = 0.0001
        
    def backward (self, y):
        # TODO: IMPLEMENT ME
        
        for i in reversed(range(self.T)):
            dJ_dy = y[i] - self.y_pred[i]  # 1x1
            self.dw += np.dot(self.hs[i+1], dJ_dy)

            dh = np.dot(self.dw, dJ_dy) + self.dhnext
            dhraw = (1 - self.hs[i+1]*self.hs[i+1]) * dh

            self.dV += np.dot(dhraw, xs[i].T)

            self.dU += np.dot(dhraw, self.hs[i].T)
            dhnext = np.dot(self.U.T, dhraw)
            
            for dparam in [self.U,self.V,self.w]:
                np.clip(dparam, -1, 1, out=dparam)

            self.U -= self.learning_rate*self.dU
            self.V -= self.learning_rate*self.dV
            self.w -= self.learning_rate*self.dw        
        pass

    def forward (self, x):
        # TODO: IMPLEMENT ME
        self.xs = x
        for i in range(self.T):
            self.z = np.dot(self.U, self.hs[i]) + np.dot(self.V, x[i]) # 6x1
            hidden = np.tanh(self.z) # 6x1
            self.y_pred[i] = np.dot(hidden.T, self.w)[0][0] # 1x1
            self.hs.append(hidden)   
        pass
    
    def RNNloss(self,y, y_pred):
        dy = np.asarray(y_pred) - np.asarray(y)
        self.loss = 0.5*np.dot(dy, dy.T)
        return self.loss
    
    def step(self,x, y):
        self.T = len(x)
        self.y_pred = [0]*self.T
        self.hs = []
        self.hs.append(self.hidden)
        
        self.forward(x)
        self.backward(y)
        loss = self.RNNloss(y, self.y_pred)
        print(loss)
        return self.y_pred, loss

In [2]:
# From https://medium.com/@erikhallstrm/hello-world-rnn-83cd7105b767
def generateData ():
    total_series_length = 50
    echo_step = 2  # 2-back task
    batch_size = 1
    x = np.random.choice(2, total_series_length, p=[0.5, 0.5])
    y = np.roll(x, echo_step)
    y[0:echo_step] = 0
    y = list(y)
    return (x, y)

In [175]:
if __name__ == "__main__":
    xs, ys = generateData()
    
    numHidden = 6
    numInput = 1
    numTimesteps = len(xs)
    rnn = RNN(numHidden, numInput, 1)
    
    # TODO: IMPLEMENT ME    
    for iter in range(100):
        xs, ys = generateData()
        y_pred, loss = rnn.step(xs, ys)

12.00339182066314
11.505430826429818
13.009855523775762
12.298281560548954
9.998301818246974
10.396689608129291
9.96819271457521
11.787758491288283
15.705192069307941
7.750082971724172
45.17658317579392
238.51509055055394
758.3681631414016
808.5559631526098
1212.0751580588442
651.4988079952105
1010.6637685127316
1014.9571736344166
1074.8208216583928
1165.036120180709
1205.985208121261
1295.8441828523657
1391.0559626822287
1465.4195549871315
1357.3475020313954
1494.6573782269265
2110.500228913083
2214.638873368896
2154.5363026289497
10.298931642103781
1834.9762009581382
14.534531274770124
15.005429688131814
10.571185959676297
13.676916054306133
14.123964398284016
16.387348313432636
13.918305196753195
16.2770544768523
16.225059788409418
21.531996050081446
13.429346626414032
18.882651969579317
23.225048984265975
19.792906711887884
15.157452374988262
22.90389328111518
23.85487644945845
26.653711061494697
27.014842164416383
24.582700995396824
25.670204658792013
32.53308722156905
29.18757146